In [1]:
import  pandas as pd
import numpy as np
import json

In [2]:
df=pd.read_csv("iris.csv")
with open("algoparams_from_ui.json.json","r") as f:
    jfile=f.read()
data=json.loads(jfile)

In [3]:
data.keys()

dict_keys(['session_name', 'session_description', 'design_state_data'])

In [4]:
data["design_state_data"].keys()

dict_keys(['session_info', 'target', 'train', 'metrics', 'feature_handling', 'feature_generation', 'feature_reduction', 'hyperparameters', 'weighting_stratergy', 'probability_calibration', 'algorithms'])

import all algorithms

step 1

In [5]:
type_=data["design_state_data"]["target"]["type"]
target=data["design_state_data"]["target"]["target"]
type_,target


('regression', 'petal_width')

step2:handling missing value

In [6]:
for i in data["design_state_data"]["feature_handling"]:
    print()
    print(i)
    print(data["design_state_data"]["feature_handling"][i]["feature_details"])           


sepal_length
{'numerical_handling': 'Keep as regular numerical feature', 'rescaling': 'No rescaling', 'make_derived_feats': False, 'missing_values': 'Impute', 'impute_with': 'Average of values', 'impute_value': 0}

sepal_width
{'numerical_handling': 'Keep as regular numerical feature', 'rescaling': 'No rescaling', 'make_derived_feats': False, 'missing_values': 'Impute', 'impute_with': 'custom', 'impute_value': -1}

petal_length
{'numerical_handling': 'Keep as regular numerical feature', 'rescaling': 'No rescaling', 'make_derived_feats': False, 'missing_values': 'Impute', 'impute_with': 'Average of values', 'impute_value': 0}

petal_width
{'numerical_handling': 'Keep as regular numerical feature', 'rescaling': 'No rescaling', 'make_derived_feats': False, 'missing_values': 'Impute', 'impute_with': 'custom', 'impute_value': -2}

species
{'text_handling': 'Tokenize and hash', 'hash_columns': 0}


In [7]:
df["sepal_length"]=df.sepal_length.fillna(df["sepal_length"].mean())
df["sepal_width"]=df.sepal_length.fillna(-1)
df["petal_length"]=df.sepal_length.fillna(df["sepal_length"].mean())
df["petal_width"]=df.sepal_length.fillna(-2)

In [8]:
df=pd.get_dummies(df,drop_first=True)
df.head(3)


,sepal_length,sepal_width,petal_length,petal_width,species_Iris-versicolor,species_Iris-virginica
0,5.1,5.1,5.1,5.1,0,0
1,4.9,4.9,4.9,4.9,0,0
2,4.7,4.7,4.7,4.7,0,0


step3:feature Reduction

In [9]:
data["design_state_data"].keys()

dict_keys(['session_info', 'target', 'train', 'metrics', 'feature_handling', 'feature_generation', 'feature_reduction', 'hyperparameters', 'weighting_stratergy', 'probability_calibration', 'algorithms'])

In [10]:
data["design_state_data"]["feature_reduction"]

{'feature_reduction_method': 'Tree-based',
 'num_of_features_to_keep': '4',
 'num_of_trees': '5',
 'depth_of_trees': '6'}

In [11]:
y=df[target]

In [12]:
#apply feature Reduction 
if data["design_state_data"]["feature_reduction"]['feature_reduction_method']=="PCA":
    from sklearn.decomposition import PCA
    pca=PCA(n_components=data["design_state_data"]["feature_reduction"]['num_of_features_to_keep'])
    x=pca.fit_transform(df.drop(target,axis=1))
if data["design_state_data"]["feature_reduction"]['feature_reduction_method']=="No Reduction":
    x=df.drop(target,axis=1)
else:
    x=df.drop(target,axis=1)


In [13]:
# Train test Split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=10)

step4: Model Building

In [14]:
data["design_state_data"]["algorithms"].keys()

dict_keys(['RandomForestClassifier', 'RandomForestRegressor', 'GBTClassifier', 'GBTRegressor', 'LinearRegression', 'LogisticRegression', 'RidgeRegression', 'LassoRegression', 'ElasticNetRegression', 'xg_boost', 'DecisionTreeRegressor', 'DecisionTreeClassifier', 'SVM', 'SGD', 'KNN', 'extra_random_trees', 'neural_network'])

In [15]:
#check which model to choose 
for i in data["design_state_data"]["algorithms"]:
    if data["design_state_data"]["algorithms"][i]["is_selected"]==True:
        print(data["design_state_data"]["algorithms"][i]["model_name"])

Random Forest Regressor


In [16]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor,GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,LogisticRegression,Ridge,Lasso,ElasticNet,SGDClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import sklearn.neural_network

In [17]:
data["design_state_data"]["algorithms"]["RandomForestRegressor"]


{'model_name': 'Random Forest Regressor',
 'is_selected': True,
 'min_trees': 10,
 'max_trees': 20,
 'feature_sampling_statergy': 'Default',
 'min_depth': 20,
 'max_depth': 25,
 'min_samples_per_leaf_min_value': 5,
 'min_samples_per_leaf_max_value': 10,
 'parallelism': 0}

In [18]:
model=RandomForestRegressor(n_estimators=15,min_samples_leaf=5,max_depth=25)
model.fit(x_train,y_train)
y_predicted=model.predict(x_test)
model.score(x_test,y_test)



0.9825719036251743

step 5:#hyper tunning using gridSearchCV 

In [19]:
#hyper tunning using gridSearchCV
data["design_state_data"]['hyperparameters']

{'stratergy': 'Grid Search',
 'shuffle_grid': True,
 'random_state': 1,
 'max_iterations': 2,
 'max_search_time': 3,
 'parallelism': 5,
 'cross_validation_stratergy': 'Time-based K-fold(with overlap)',
 'num_of_folds': 6,
 'split_ratio': 0,
 'stratified': True}

In [20]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(model,{"n_estimators":[40,50,100]},cv=6)
clf=clf.fit(x_train,y_train)


In [21]:
score= pd.DataFrame(clf.cv_results_)
score

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,mean_test_score,std_test_score,rank_test_score
0,0.036096,0.005927,0.003137,0.005721,40,{'n_estimators': 40},0.992164,0.952122,0.995426,0.993862,0.958851,0.993555,0.980997,0.018168,1
1,0.037539,0.008811,0.003396,0.007594,50,{'n_estimators': 50},0.987606,0.951706,0.995656,0.991176,0.958073,0.992153,0.979395,0.017582,3
2,0.085063,0.008402,0.006367,0.003659,100,{'n_estimators': 100},0.989119,0.951198,0.996902,0.993793,0.958527,0.993117,0.980443,0.018351,2


In [22]:
clf.best_score_

0.9809966536100615

step 6:Log to the console the standard model metrics that apply